In [ ]:
from qch2025.pkg.models.RNN import network as network
from qch2025.pkg.models.RNN import func as func

from qch2025.pkg.dataset import DS

import qch2025.pkg.plotting as plt

import numpy as np
import pandas as pd
import torch
import datetime

n_epochs = 50
window_size = 256
batch_size = 32
window_steps = 16
attn_heads = 4

learning_rate = 0.0001
dropout = 0.2

r2_priority = 0.1

dtype = torch.float32

# Training
train_dataset = DS(dataset_path="/home/ubuntu/repos/quantchallenge-2025/qch2025/dataset/train.csv",
                        window_size=window_size,
                        window_steps=window_steps,
                        dtype=dtype)

rnn = network.RNN(14, 512, 2, dropout, attn_heads, 2, device=torch.device("cuda"), dtype=dtype)
losses = func.train(rnn, dataset=train_dataset, epochs=n_epochs, learning_rate=learning_rate, decay=r2_priority)

plt.plot_line(np.arange(1, n_epochs+1), losses)


#checkpoint = torch.load("/home/ubuntu/repos/quantchallenge-2025/weights/weights.pth", weights_only=True)
#rnn.load_state_dict(checkpoint['model_state_dict'])

eval = DS(dataset_path="/home/ubuntu/repos/quantchallenge-2025/qch2025/dataset/test.csv",
                    window_size=window_size,
                    window_steps=1,
                    eval=True,
                    device=torch.device("cuda"),
                    dtype=dtype)
print(eval.ids.shape, eval.train.shape)
y1, y2, ids = func.eval(rnn, eval)

df = pd.DataFrame({"id": ids, "Y1": y1, "Y2": y2})
df["id"] = df["id"] + 1
df.to_csv(f"/home/ubuntu/repos/quantchallenge-2025/qch2025/outputs/{datetime.datetime.now()}_predicted.csv")






torch.Size([80000, 14])
torch.Size([80000, 2])
Initialized dataset torch.Size([4484, 256, 14]) torch.Size([4484, 256, 2])
[2025-09-21 11:53:37.823917] Current batch item: 0, took 236 ms, loss: 0.6871525049209595, mean loss: 0.6871525049209595
Saving checkpoint: best rsq: 0.2878805476102831 vs current: 0
1: loss mean = 0.2878805476102831, RSQ: 0.2878805476102831
[2025-09-21 11:53:44.880568] Current batch item: 0, took 40 ms, loss: 0.41733941435813904, mean loss: 0.41733941435813904
Saving checkpoint: best rsq: 0.6417324522726744 vs current: 0.2878805476102831


KeyboardInterrupt: 